In [1]:
!pip install pytorch-transformers

     |████████████████████████████████| 184kB 3.0MB/s 
     |████████████████████████████████| 675kB 45.4MB/s 
     |████████████████████████████████| 860kB 45.7MB/s 
     |████████████████████████████████| 1.0MB 43.4MB/s 
  Created wheel for regex: filename=regex-2019.12.9-cp36-cp36m-linux_x86_64.whl size=609185 sha256=8267ea5efd3b67f8f3855e719db9f3c6e22aa71611e64cc03fb5cd553436f030
  Stored in directory: /root/.cache/pip/wheels/0d/fb/b3/a89169557229468c49ca64f6839418f22461f6ee0a74f342b1
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=9b9eb55d1f7001f5ff6ba714a7dff226237868e1c1db9cb7156c527558b674b4
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built regex sacremoses


In [0]:
import pandas as pd
import itertools
import torch
from pytorch_transformers import BertTokenizer, BertModel, BertForMaskedLM

In [0]:
import numpy as np

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

100%|██████████| 231508/231508 [00:00<00:00, 6110355.54B/s]


In [5]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

100%|██████████| 440473133/440473133 [00:06<00:00, 70278327.66B/s]


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [10]:
word_sentence_pair = pd.read_csv("train_masked_new_final.csv")
word_sentence_pair.head()

,Unnamed: 0,id,special,whole,label,masked
0,0,0,orange juice,he pour orange juice on his cereal,0,he pour [MASK] on his cereal
1,1,0,milk,he pour milk on his cereal,1,he pour [MASK] on his cereal
2,2,1,apple,he drink apple,0,he drink [MASK]
3,3,1,milk,he drink milk,1,he drink [MASK]
4,4,2,a,jeff run a mile today,1,jeff run [MASK] mile today


In [11]:
len(word_sentence_pair['id'].unique())

9992

In [14]:
#id_list = []
predicted_result = {}
#true_result = []
for i in range(2000,10000):
  #i = 1999
  
  try_data = word_sentence_pair[word_sentence_pair.id.values == i]
  if try_data.shape[0] ==2:
    #try_data = word_sentence_pair[word_sentence_pair.id.values == 1999]
    special_0 = try_data.special.values[try_data.label == 0]
    special_1 = try_data.special.values[try_data.label == 1]
    tokenized_text1 = []
    tokenized_text0 = []
    for j in range(len(special_1)):
      tokenized_text1.append(tokenizer.tokenize(special_1[j]))
    #print("tokenized_text1: ", tokenized_text1)
    for k in range(len(special_0)):
      tokenized_text0.append(tokenizer.tokenize(special_0[k]))
    #print("tokenized_text0: ", tokenized_text0)
    if len(tokenized_text1) == 0 or len(tokenized_text0) == 0:
      print("has error in special word for sample: ", try_data.id.values)
      predicted_result[i] = "NA"
      continue
    sentence_structure = try_data.masked[try_data.label == 0].values
    sent = "[CLS] " + sentence_structure[0] + " [SEP]"
    tokenized_structure = tokenizer.tokenize(sent)
    masked_indx = [iii for iii, x in enumerate(tokenized_structure) if x == '[MASK]']
    # Convert token to vocabulary indices
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_structure)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])

    with torch.no_grad():
      #print(tokens_tensor)
      outputs = model(tokens_tensor)
      predictions = outputs[0]
    idx_special_0 = tokenizer.convert_tokens_to_ids(tokenized_text0[0])
    idx_special_1 = tokenizer.convert_tokens_to_ids(tokenized_text1[0])
    #print("masked_indx: ",masked_indx)
    #print("idx_special_0:", idx_special_0)
    #print(predictions[0,masked_indx][0])
    score_0 = sum(predictions[0,masked_indx][0][idx_special_0])/len(idx_special_0)
    score_1 = sum(predictions[0,masked_indx][0][idx_special_1])/len(idx_special_1)
    #print("score 0: ", score_0)
    #print("score 1: ", score_1)
    score = [score_0, score_1]
    predicted_label = score.index(max(score))
    #print(predicted_label)
    #predicted_result.append(predicted_label)
    #if predicted_label != 1:
      #print("this sample is wrong: ", try_data.id.values)
  else:
    #try_data = word_sentence_pair[word_sentence_pair.id.values == 17]
    special_0 = try_data.special.values[try_data.label == 0] # a list of special words
    special_1 = try_data.special.values[try_data.label == 1]
    tokenized_text1 = []
    tokenized_text0 = []
    for j in range(len(special_1)):
      tokenized_text1.append(tokenizer.tokenize(special_1[j]))
    #print("tokenized_text1: ", tokenized_text1)
    for k in range(len(special_0)):
      tokenized_text0.append(tokenizer.tokenize(special_0[k]))  
    #print("tokenized_text0: ", tokenized_text0)
    idx_special_0 = []
    idx_special_1 = []
    for tocken in tokenized_text0:
      idx_special_0.append(tokenizer.convert_tokens_to_ids(tocken))
    #idx_special_0 = [tokenizer.convert_tokens_to_ids(spec_idx for spec_idx in tokenized_text0)]
    #idx_special_0 = [tokenizer.convert_tokens_to_ids(spec_idx for spec_idx in tokenized_text0)]
    for tocken in tokenized_text1:
      idx_special_1.append(tokenizer.convert_tokens_to_ids(tocken))
    #idx_special_1 = tokenizer.convert_tokens_to_ids(tokenized_text1[0])

    # for score 0
    #masked_indx_0 = []
    score_0 = 0
    sentence_structure_0 = try_data.masked[try_data.label == 0].values # many be two different 
    #print(sentence_structure_0)
    for struct_idx in range(len(sentence_structure_0)):
      sent = "[CLS] " + sentence_structure_0[struct_idx] + " [SEP]"
      #print(sent)
      tokenized_structure = tokenizer.tokenize(sent)
      masked_indx = [iii for iii, x in enumerate(tokenized_structure) if x == '[MASK]']
      #masked_indx_0.append(masked_indx)
      #print(masked_indx, 'masked index')
    # Convert token to vocabulary indices
      indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_structure)

    # Convert inputs to PyTorch tensors
      tokens_tensor = torch.tensor([indexed_tokens])

      with torch.no_grad():
      #print(tokens_tensor)
        outputs = model(tokens_tensor)
        predictions = outputs[0]
        #print(predictions)
      score_0 += sum(predictions[0,masked_indx][0][idx_special_0[struct_idx]])/len(idx_special_0[struct_idx])
    #print("score 0: ", score_0)
      #score_0 = score_0


    # for score 1
    #masked_indx_0 = []
    score_1 = 0
    sentence_structure_1 = try_data.masked[try_data.label == 1].values # many be two different 
    #print(sentence_structure_0)
    for struct_idx in range(len(sentence_structure_1)):
      sent = "[CLS] " + sentence_structure_1[struct_idx] + " [SEP]"
      #print(sent)
      tokenized_structure = tokenizer.tokenize(sent)
      masked_indx = [iii for iii, x in enumerate(tokenized_structure) if x == '[MASK]']
      #masked_indx_0.append(masked_indx)
      #print(masked_indx, 'masked index')
    # Convert token to vocabulary indices
      indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_structure)

    # Convert inputs to PyTorch tensors
      tokens_tensor = torch.tensor([indexed_tokens])

      with torch.no_grad():
      #print(tokens_tensor)
        outputs = model(tokens_tensor)
        predictions = outputs[0]
        #print(predictions)
      score_1 += sum(predictions[0,masked_indx][0][idx_special_1[struct_idx]])/len(idx_special_1[struct_idx])
    #print("score 1: ",score_1)
    score = [score_0, score_1]
    predicted_label = score.index(max(score))
  print(predicted_label, " is predicted result")
  predicted_result[i] = [score_0,score_1,predicted_label]
  if predicted_label != 1:
    print("this sample is wrong: ", try_data.id.values)

1  is predicted result
0  is predicted result
this sample is wrong:  [2001 2001]
0  is predicted result
this sample is wrong:  [2002 2002]
1  is predicted result
1  is predicted result
0  is predicted result
this sample is wrong:  [2005 2005]
1  is predicted result
1  is predicted result
1  is predicted result
0  is predicted result
this sample is wrong:  [2009 2009]
1  is predicted result
1  is predicted result
1  is predicted result
1  is predicted result
1  is predicted result
1  is predicted result
1  is predicted result
1  is predicted result
0  is predicted result
this sample is wrong:  [2018 2018 2018]
1  is predicted result
0  is predicted result
this sample is wrong:  [2020 2020 2020]
1  is predicted result
1  is predicted result
1  is predicted result
0  is predicted result
this sample is wrong:  [2024 2024]
1  is predicted result
1  is predicted result
1  is predicted result
0  is predicted result
this sample is wrong:  [2028 2028 2028 2028 2028 2028]
1  is predicted result


In [15]:
predicted_result[len(predicted_result)-1]

[tensor(1.7923), tensor(0.0009), 0]

In [16]:
len(predicted_result)

8000

In [24]:
predicted_result

{2000: [tensor(-0.7254), tensor(8.2250), 1],
 2001: [tensor(-1.9649), tensor(-2.6955), 0],
 2002: [tensor(4.1957), tensor(1.8079), 0],
 2003: [tensor(1.8884), tensor(7.3425), 1],
 2004: [tensor(0.9541), tensor(10.9659), 1],
 2005: [tensor(5.1427), tensor(4.3223), 0],
 2006: [tensor(-3.3327), tensor(4.0402), 1],
 2007: [tensor(-0.9567), tensor(12.3863), 1],
 2008: [tensor(-2.3009), tensor(2.1268), 1],
 2009: [tensor(-0.3910), tensor(-0.5362), 0],
 2010: [tensor(-0.2563), tensor(1.1313), 1],
 2011: [tensor(-3.5905), tensor(2.6354), 1],
 2012: [tensor(-5.2662), tensor(-2.5593), 1],
 2013: [tensor(0.6689), tensor(2.7220), 1],
 2014: [0, tensor(6.7016), 1],
 2015: [tensor(-0.7980), tensor(5.3536), 1],
 2016: [tensor(6.6491), tensor(7.8993), 1],
 2017: [tensor(-2.6199), tensor(0.0999), 1],
 2018: [tensor(20.2053), tensor(-1.0400), 0],
 2019: [tensor(-0.5152), tensor(0.4301), 1],
 2020: [tensor(4.8809), tensor(2.4113), 0],
 2021: [tensor(3.1623), tensor(4.8115), 1],
 2022: [tensor(2.3123), te

In [0]:
sum(predicted_result)/len(predicted_result)

0.6816076123701014

In [0]:
result_dict_train = {}
for i in range(len(predicted_result)):
  result_dict_train[i] = predicted_result[i+2000]

In [22]:
result_dict_train

{0: [tensor(-0.7254), tensor(8.2250), 1],
 1: [tensor(-1.9649), tensor(-2.6955), 0],
 2: [tensor(4.1957), tensor(1.8079), 0],
 3: [tensor(1.8884), tensor(7.3425), 1],
 4: [tensor(0.9541), tensor(10.9659), 1],
 5: [tensor(5.1427), tensor(4.3223), 0],
 6: [tensor(-3.3327), tensor(4.0402), 1],
 7: [tensor(-0.9567), tensor(12.3863), 1],
 8: [tensor(-2.3009), tensor(2.1268), 1],
 9: [tensor(-0.3910), tensor(-0.5362), 0],
 10: [tensor(-0.2563), tensor(1.1313), 1],
 11: [tensor(-3.5905), tensor(2.6354), 1],
 12: [tensor(-5.2662), tensor(-2.5593), 1],
 13: [tensor(0.6689), tensor(2.7220), 1],
 14: [0, tensor(6.7016), 1],
 15: [tensor(-0.7980), tensor(5.3536), 1],
 16: [tensor(6.6491), tensor(7.8993), 1],
 17: [tensor(-2.6199), tensor(0.0999), 1],
 18: [tensor(20.2053), tensor(-1.0400), 0],
 19: [tensor(-0.5152), tensor(0.4301), 1],
 20: [tensor(4.8809), tensor(2.4113), 0],
 21: [tensor(3.1623), tensor(4.8115), 1],
 22: [tensor(2.3123), tensor(2.5091), 1],
 23: [tensor(-1.0392), tensor(-0.6991)

In [0]:
import pickle

In [0]:
with open("masked_model_prediction_2000_10000_with_scores.pickle","wb") as handle:
  pickle.dump(predicted_result,handle)

In [0]:
len(predicted_result)

2000

In [0]:
indices = [i for i, x in enumerate(predicted_result) if x == 'need more computation']

In [0]:
indices

[5,
 17,
 20,
 34,
 37,
 41,
 43,
 46,
 54,
 56,
 71,
 72,
 75,
 81,
 85,
 89,
 90,
 91,
 93,
 96,
 98,
 100,
 103,
 105,
 107,
 110,
 114,
 123,
 125,
 132,
 137,
 157,
 167,
 178,
 182,
 191,
 195,
 200,
 206,
 218,
 223,
 227,
 229,
 232,
 248,
 252,
 253,
 256,
 258,
 259,
 266,
 276,
 282,
 290,
 293,
 304,
 308,
 324,
 329,
 332,
 356,
 358,
 372,
 375,
 376,
 379,
 380,
 386,
 388,
 389,
 390,
 406,
 409,
 411,
 414,
 416,
 430,
 434,
 440,
 448,
 458,
 459,
 488,
 497,
 522,
 528,
 543,
 553,
 558,
 561,
 563,
 566,
 569,
 578,
 591,
 595,
 596,
 597,
 598,
 603,
 606,
 611,
 624,
 634,
 682,
 686,
 700,
 728,
 735,
 741,
 747,
 749,
 754,
 806,
 813,
 820,
 833,
 858,
 871,
 873,
 890,
 895,
 912,
 918,
 928,
 1005,
 1023,
 1029,
 1030,
 1031,
 1035,
 1046,
 1047,
 1056,
 1059,
 1064,
 1081,
 1085,
 1096,
 1101,
 1112,
 1114,
 1116,
 1133,
 1145,
 1149,
 1153,
 1171,
 1197,
 1220,
 1240,
 1252,
 1268,
 1272,
 1292,
 1305,
 1308,
 1333,
 1338,
 1341,
 1352,
 1353,
 1354,
 1357,

In [0]:
sum(predicted_result)/len(predicted_result)

TypeError: ignored

In [0]:
try_data = word_sentence_pair[word_sentence_pair.id.values == 17]
try_data

,Unnamed: 0,id,special,whole,label,masked
36,36,17,moses,lisa play jazz music for moses of the old test...,0,lisa play jazz music for [MASK] of the old tes...
37,37,17,old testament,lisa play jazz music for moses of the old test...,0,lisa play jazz music for moses of the [MASK] y...
38,38,17,the president,lisa play jazz music for the president of the ...,1,lisa play jazz music for [MASK] of the usa yes...
39,39,17,usa,lisa play jazz music for the president of the ...,1,lisa play jazz music for the president of the ...


In [0]:
try_data = word_sentence_pair[word_sentence_pair.id.values == 17]
special_0 = try_data.special.values[try_data.label == 0] # a list of special words
special_1 = try_data.special.values[try_data.label == 1]
tokenized_text1 = []
tokenized_text0 = []
for j in range(len(special_1)):
  tokenized_text1.append(tokenizer.tokenize(special_1[j]))
print("tokenized_text1: ", tokenized_text1)
for k in range(len(special_0)):
  tokenized_text0.append(tokenizer.tokenize(special_0[k]))  
print("tokenized_text0: ", tokenized_text0)
idx_special_0 = []
idx_special_1 = []
for tocken in tokenized_text0:
  idx_special_0.append(tokenizer.convert_tokens_to_ids(tocken))
#idx_special_0 = [tokenizer.convert_tokens_to_ids(spec_idx for spec_idx in tokenized_text0)]
#idx_special_0 = [tokenizer.convert_tokens_to_ids(spec_idx for spec_idx in tokenized_text0)]
for tocken in tokenized_text1:
  idx_special_1.append(tokenizer.convert_tokens_to_ids(tocken))
#idx_special_1 = tokenizer.convert_tokens_to_ids(tokenized_text1[0])

# for score 0
#masked_indx_0 = []
score_0 = 0
sentence_structure_0 = try_data.masked[try_data.label == 0].values # many be two different 
#print(sentence_structure_0)
for struct_idx in range(len(sentence_structure_0)):
  sent = "[CLS] " + sentence_structure_0[struct_idx] + " [SEP]"
  print(sent)
  tokenized_structure = tokenizer.tokenize(sent)
  masked_indx = [iii for iii, x in enumerate(tokenized_structure) if x == '[MASK]']
  #masked_indx_0.append(masked_indx)
  print(masked_indx, 'masked index')
# Convert token to vocabulary indices
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_structure)

# Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])

  with torch.no_grad():
  #print(tokens_tensor)
    outputs = model(tokens_tensor)
    predictions = outputs[0]
    #print(predictions)
  score_0 += sum(predictions[0,masked_indx][0][idx_special_0[struct_idx]])/len(idx_special_0[struct_idx])
print("score 0: ", score_0)
  #score_0 = score_0


# for score 1
#masked_indx_0 = []
score_1 = 0
sentence_structure_1 = try_data.masked[try_data.label == 1].values # many be two different 
#print(sentence_structure_0)
for struct_idx in range(len(sentence_structure_1)):
  sent = "[CLS] " + sentence_structure_1[struct_idx] + " [SEP]"
  print(sent)
  tokenized_structure = tokenizer.tokenize(sent)
  masked_indx = [iii for iii, x in enumerate(tokenized_structure) if x == '[MASK]']
  #masked_indx_0.append(masked_indx)
  print(masked_indx, 'masked index')
# Convert token to vocabulary indices
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_structure)

# Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])

  with torch.no_grad():
  #print(tokens_tensor)
    outputs = model(tokens_tensor)
    predictions = outputs[0]
    #print(predictions)
  score_1 += sum(predictions[0,masked_indx][0][idx_special_1[struct_idx]])/len(idx_special_1[struct_idx])
print("score 1: ",score_1)


    #print("masked_indx: ",masked_indx)
#print("idx_special_0:", idx_special_0)
#print("idx_special_1:", idx_special_1)
#print(masked_indx)
#print(predictions[0,masked_indx])
#score_0 = sum(predictions[0,masked_indx][0][idx_special_0])/len(idx_special_0)
#score_1 = sum(predictions[0,masked_indx][0][idx_special_1])/len(idx_special_1)
#print("score 0: ", score_0)
#print("score 1: ", score_1)
score = [score_0, score_1]
predicted_label = score.index(max(score))
#print(predicted_label)
    #predicted_result.append(predicted_label)

tokenized_text1:  [['the', 'president'], ['usa']]
tokenized_text0:  [['moses'], ['old', 'testament']]
[CLS] lisa play jazz music for [MASK] of the old testament yesterday [SEP]
[6] masked index
[CLS] lisa play jazz music for moses of the [MASK] yesterday [SEP]
[9] masked index
score 0:  tensor(5.9146)
[CLS] lisa play jazz music for [MASK] of the usa yesterday [SEP]
[6] masked index
[CLS] lisa play jazz music for the president of the [MASK] yesterday [SEP]
[10] masked index
score 1:  tensor(11.8656)


In [0]:
predicted_label

1

In [0]:
sum(predictions[0,masked_indx][0][[3328,1999]])/2

tensor(1.1595)

In [0]:
predictions[0,[2]][0][3671]

tensor(16.2562)

In [0]:
masked_indx_0

[[2]]

In [0]:
idx_special_0

[[3671], [3328, 1999]]

In [0]:
idx_special_1

[[3328, 1999], [3671]]

In [0]:
true_result[8630]

0

In [0]:
import itertools
a = [['a','b'], ['c']]
print(list(itertools.chain.from_iterable(idx_special_0)))

[3671, 3328, 1999]


In [0]:
try_data = word_sentence_pair[word_sentence_pair.id.values == 0]
special_for_sent0 = try_data.special[try_data.label.values == 0].values
special_for_sent1 = try_data.special.values[try_data.label.values == 1]
sentence0 = "[CLS] " + try_data.whole.values[try_data.label.values == 0] + " [SEP]"  
sentence1 = "[CLS] " + try_data.whole.values[try_data.label.values == 1] + " [SEP]" 

print(special_for_sent0)
print(special_for_sent1)
print(sentence0)
print(sentence1)

In [0]:
special_for_sent1_tokenized = tokenizer.tokenize(special_for_sent1[0])
print(special_for_sent1_tokenized)
sent1_tokenized = tokenizer.tokenize(sentence1[0])
print(sent1_tokenized)

In [0]:
masked_index = sent1_tokenized.index(special_for_sent1_tokenized[0])
print(masked_index)

In [0]:
sent1_tokenized[masked_index] = "[MASK]"
print(sent1_tokenized)

In [0]:
# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(sent1_tokenized)

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])


In [0]:
with torch.no_grad():
    #print(tokens_tensor)
    outputs = model(tokens_tensor)
    predictions = outputs[0]